In [0]:
rdd1=sc.textFile("/FileStore/tables/Skip_Lines.txt")
rdd1.collect()
rdd2=rdd1.zipWithIndex().filter(lambda a:a[1]>2).map(lambda a:a[0].split(","))
columns=rdd2.first()
 
rdd2.filter(lambda a:a!=columns).toDF(columns).show()


#df1=spark.read.format("text").option("header",True).option("path","/FileStore/tables/Skip_Lines.txt").load()
#df1.show(truncate=False)

+---+------+-----+
| id|  name|  loc|
+---+------+-----+
|  1|Aniket| Pune|
|  2|   Raj|Dubai|
|  3|   Ram|  Usa|
+---+------+-----+



In [0]:
from pyspark.sql.functions import *
df=spark.read.format("csv").option("header",True).option("inferschema",True).option("nullValue","null").load("/FileStore/tables/emp.csv")
df.drop("UPDATED_DATE").show()
df.printSchema()
df2=df.withColumn("HIREDATE",to_date("HIREDATE",'dd-MM-yyyy')).fillna({"HIREDATE":'2000-12-31'})
df3=df2.withColumn("Year",date_format("HIREDATE",'yyyy')).withColumn("Month",date_format("HIREDATE",'MM'))
df3.printSchema()
df3.show()

+-----+------+---------+----+----------+----+----+------+
|EMPNO| ENAME|      JOB| MGR|  HIREDATE| SAL|COMM|DEPTNO|
+-----+------+---------+----+----------+----+----+------+
| 7369| SMITH|    CLERK|7902|17-12-1980| 800|null|    20|
| 7499| ALLEN| SALESMAN|7698|20-02-1981|1600| 300|    30|
| 7521|  WARD| SALESMAN|7698|22-02-1981|1250| 500|    30|
| 7566| JONES|  MANAGER|7839|04-02-1981|2975|null|    20|
| 7654|MARTIN| SALESMAN|7698|21-09-1981|1250|1400|    30|
| 7698|   SGR|  MANAGER|7839|05-01-1981|2850|null|    30|
| 7782|  RAVI|  MANAGER|7839|06-09-1981|2450|null|    10|
| 7788| SCOTT|  ANALYST|7566|19-04-1987|3000|null|    20|
| 7839|  KING|PRESIDENT|null|01-11-1981|5000|null|    10|
| 7844|TURNER| SALESMAN|7698|09-08-1981|1500|   0|    30|
| 7876| ADAMS|    CLERK|7788|23-05-1987|1100|null|    20|
| 7900| JAMES|    CLERK|7698|12-03-1981| 950|null|    30|
| 7902|  FORD|  ANALYST|7566|12-03-1981|3000|null|    20|
| 7934|MILLER|    CLERK|7782|01-03-1982|1300|null|    10|
| 1234|SEKHAR|

In [0]:
dbutils.fs.put("/FileStore/tables/varying_columns_new1.csv",""" 1,Ravi
2,Aniket,Pune
3,Ram,Satara,India
4,Raj,Kolhapur,India,raj@xyz.com
""")



Wrote 74 bytes.
+--------------------+
|               value|
+--------------------+
|              1,Ravi|
|       2,Aniket,Pune|
|  3,Ram,Satara,India|
|4,Raj,Kolhapur,In...|
+--------------------+



In [0]:
from pyspark.sql.functions import *
df11=spark.read.format("text").option("path","/FileStore/tables/varying_columns_new1.csv").load
#If we use format as csv then only data of 2 columns willbe displayed as (no of columns in dataframe =no of columns in first row).Hence use text format

df11.show(truncate=False)
df12=df11.withColumn("splittable_col",split("value",","))
#df12.select("splittable_col",size("splittable_col").alias("size")).show()
max_value=df12.select(max(size("splittable_col"))).collect()[0][0]
print(max_value)
for i in range(0,max_value):
    df12=df12.withColumn("col"+str(i),df12["splittable_col"][i])
df12.show()

+--------------------------------+
|value                           |
+--------------------------------+
| 1,Ravi                         |
|2,Aniket,Pune                   |
|3,Ram,Satara,India              |
|4,Raj,Kolhapur,India,raj@xyz.com|
+--------------------------------+

5
+--------------------+--------------------+----+------+--------+-----+-----------+
|               value|      splittable_col|col0|  col1|    col2| col3|       col4|
+--------------------+--------------------+----+------+--------+-----+-----------+
|              1,Ravi|          [ 1, Ravi]|   1|  Ravi|    null| null|       null|
|       2,Aniket,Pune|   [2, Aniket, Pune]|   2|Aniket|    Pune| null|       null|
|  3,Ram,Satara,India|[3, Ram, Satara, ...|   3|   Ram|  Satara|India|       null|
|4,Raj,Kolhapur,In...|[4, Raj, Kolhapur...|   4|   Raj|Kolhapur|India|raj@xyz.com|
+--------------------+--------------------+----+------+--------+-----+-----------+



In [0]:
#If we want to include filename in the dataframe use input_file_name()

In [0]:
dbutils.fs.put("/FileStore/tables/BadRecords.csv","""CHANNEL_ID,CHANNEL_DESC,CHANNEL_CLASS,CHANNEL_CLASS_ID,CHANNEL_TOTAL,CHANNEL_TOTAL_ID
3,Direct Sales,Direct,12,Channel total,1
9,Tele Sales,Direct,12,Channel total,1
5,Catalog,Indirect,13,Channel total,1
4,Internet,Indirect,13,Channel total,1
2,Partners,Others,14,Channel total,1,Ram,KP
10 Partners Others 14 Channel total 1
11 Partners Others 14 Channel total 1
""")



Wrote 363 bytes.
+--------------------+------------+-------------+----------------+-------------+----------------+
|          CHANNEL_ID|CHANNEL_DESC|CHANNEL_CLASS|CHANNEL_CLASS_ID|CHANNEL_TOTAL|CHANNEL_TOTAL_ID|
+--------------------+------------+-------------+----------------+-------------+----------------+
|                   3|Direct Sales|       Direct|              12|Channel total|               1|
|                   9|  Tele Sales|       Direct|              12|Channel total|               1|
|                   5|     Catalog|     Indirect|              13|Channel total|               1|
|                   4|    Internet|     Indirect|              13|Channel total|               1|
|                   2|    Partners|       Others|              14|Channel total|               1|
|10 Partners Other...|        null|         null|            null|         null|            null|
|11 Partners Other...|        null|         null|            null|         null|            null|
+--

In [0]:
from pyspark.sql.types import *
new_schema=StructType([
    StructField("CHANNEL_ID",IntegerType(),True),
    StructField("CHANNEL_DESC",StringType(),True),
    StructField("CHANNEL_CLASS",StringType(),True),
    StructField("CHANNEL_CLASS_ID",IntegerType(),True),
    StructField("CHANNEL_TOTAL",StringType(),True),
    StructField("CHANNEL_TOTAL_ID",StringType(),True),
    StructField("BAD_RECORD",StringType(),True)
])
df_new=spark.read.format("csv").schema(new_schema).option("mode","PERMISSIVE").option("header",True).option("path","/FileStore/tables/BadRecords.csv") \
.option("columnNameOfCorruptRecord","BAD_RECORD").load()
df_new.show(truncate=False)
df_new.cache()
good_data=df_new.filter("BAD_RECORD is null").drop("BAD_RECORD")
good_data.show()
bad_data=df_new.filter("BAD_RECORD is not null").select("BAD_RECORD")
bad_data.show()

+----------+------------+-------------+----------------+-------------+----------------+-------------------------------------------+
|CHANNEL_ID|CHANNEL_DESC|CHANNEL_CLASS|CHANNEL_CLASS_ID|CHANNEL_TOTAL|CHANNEL_TOTAL_ID|BAD_RECORD                                 |
+----------+------------+-------------+----------------+-------------+----------------+-------------------------------------------+
|3         |Direct Sales|Direct       |12              |Channel total|1               |null                                       |
|9         |Tele Sales  |Direct       |12              |Channel total|1               |null                                       |
|5         |Catalog     |Indirect     |13              |Channel total|1               |null                                       |
|4         |Internet    |Indirect     |13              |Channel total|1               |null                                       |
|2         |Partners    |Others       |14              |Channel total|1     

In [0]:
df19=spark.read.format("csv").schema(new_schema).option("badRecordsPath","/FileStore/badrecords").option("header",True).option("path","/FileStore/tables/BadRecords.csv").load()
df19.show()


+----------+------------+-------------+----------------+-------------+----------------+
|CHANNEL_ID|CHANNEL_DESC|CHANNEL_CLASS|CHANNEL_CLASS_ID|CHANNEL_TOTAL|CHANNEL_TOTAL_ID|
+----------+------------+-------------+----------------+-------------+----------------+
|         3|Direct Sales|       Direct|              12|Channel total|               1|
|         9|  Tele Sales|       Direct|              12|Channel total|               1|
|         5|     Catalog|     Indirect|              13|Channel total|               1|
|         4|    Internet|     Indirect|              13|Channel total|               1|
+----------+------------+-------------+----------------+-------------+----------------+



In [0]:
#%fs ls /FileStore/badrecords/20221119T073103/bad_records
bad_data_df=spark.read.json("FileStore/badrecords/*/*")
display(bad_data_df)

path,reason,record
dbfs:/FileStore/tables/BadRecords.csv,org.apache.spark.sql.catalyst.csv.MalformedCSVException: Malformed CSV record,"2,Partners,Others,14,Channel total,1,Ram,KP"
dbfs:/FileStore/tables/BadRecords.csv,org.apache.spark.sql.catalyst.csv.MalformedCSVException: Malformed CSV record,10 Partners Others 14 Channel total 1
dbfs:/FileStore/tables/BadRecords.csv,org.apache.spark.sql.catalyst.csv.MalformedCSVException: Malformed CSV record,11 Partners Others 14 Channel total 1
